In [1]:
import torch
torch.manual_seed(1)
import numpy as np
import csv
import matplotlib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('Names.csv')

namecol = data.iloc[:,1]
countcol = data.iloc[:,4]
total_list = {}
for name, count in zip(namecol, countcol):
    if name in total_list:
        total_list[name] += count 
    else:
        total_list[name] = count


final_list = [[name, total] for name, total in total_list.items()]


In [3]:
final_list

[['Mary', 4130441],
 ['Anna', 876488],
 ['Emma', 595546],
 ['Elizabeth', 1606282],
 ['Minnie', 159352],
 ['Margaret', 1243750],
 ['Ida', 186360],
 ['Alice', 551034],
 ['Bertha', 208444],
 ['Sarah', 1063944],
 ['Annie', 348017],
 ['Clara', 268980],
 ['Ella', 273663],
 ['Florence', 335885],
 ['Cora', 122036],
 ['Martha', 549157],
 ['Laura', 794233],
 ['Nellie', 149321],
 ['Grace', 469034],
 ['Carrie', 266508],
 ['Maude', 45594],
 ['Mabel', 132785],
 ['Bessie', 170733],
 ['Jennie', 126902],
 ['Gertrude', 177329],
 ['Julia', 447836],
 ['Hattie', 97702],
 ['Edith', 264833],
 ['Mattie', 131001],
 ['Rose', 477271],
 ['Catherine', 655560],
 ['Lillian', 421392],
 ['Ada', 91806],
 ['Lillie', 154998],
 ['Helen', 1019006],
 ['Jessie', 274931],
 ['Louise', 333948],
 ['Ethel', 280116],
 ['Lula', 84353],
 ['Myrtle', 136933],
 ['Eva', 252465],
 ['Frances', 592216],
 ['Lena', 134010],
 ['Lucy', 185064],
 ['Edna', 294989],
 ['Maggie', 110740],
 ['Pearl', 158506],
 ['Daisy', 135454],
 ['Fannie', 85079],


In [4]:
#randomaf normalized weights for char pair in the future
final_listx = final_list
for line in final_listx:
    line[1]=line[1]/100000
final_listx

[['Mary', 41.30441],
 ['Anna', 8.76488],
 ['Emma', 5.95546],
 ['Elizabeth', 16.06282],
 ['Minnie', 1.59352],
 ['Margaret', 12.4375],
 ['Ida', 1.8636],
 ['Alice', 5.51034],
 ['Bertha', 2.08444],
 ['Sarah', 10.63944],
 ['Annie', 3.48017],
 ['Clara', 2.6898],
 ['Ella', 2.73663],
 ['Florence', 3.35885],
 ['Cora', 1.22036],
 ['Martha', 5.49157],
 ['Laura', 7.94233],
 ['Nellie', 1.49321],
 ['Grace', 4.69034],
 ['Carrie', 2.66508],
 ['Maude', 0.45594],
 ['Mabel', 1.32785],
 ['Bessie', 1.70733],
 ['Jennie', 1.26902],
 ['Gertrude', 1.77329],
 ['Julia', 4.47836],
 ['Hattie', 0.97702],
 ['Edith', 2.64833],
 ['Mattie', 1.31001],
 ['Rose', 4.77271],
 ['Catherine', 6.5556],
 ['Lillian', 4.21392],
 ['Ada', 0.91806],
 ['Lillie', 1.54998],
 ['Helen', 10.19006],
 ['Jessie', 2.74931],
 ['Louise', 3.33948],
 ['Ethel', 2.80116],
 ['Lula', 0.84353],
 ['Myrtle', 1.36933],
 ['Eva', 2.52465],
 ['Frances', 5.92216],
 ['Lena', 1.3401],
 ['Lucy', 1.85064],
 ['Edna', 2.94989],
 ['Maggie', 1.1074],
 ['Pearl', 1.585

In [14]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(namecol).lower())))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
